In [10]:
import pandas as pd
from utils.Dataframe_tools import PSD 
from utils import add_function_to_task,process_dir,multi_PSD,write_new_PSD

In [2]:
#Setting the Values
IO='input files\KPBom.xlsx'

find_remove=[{
    '91840054':'99717392',
    '91840056':'99717485',
    '98441969':'92758523',
    '96698939':'92758528',     # in KPbom.xlsx 
    '96698968':'92758121',     # in KPbom.xlsx
    '96768950':'92758767',     # in KPbom.xlsx
    '99413027':'92758120',
    '1020-3/4':'1020-5/6',
    '1020-3_4':'1020-5_6',
    'Desc-1020-3_4-KP_4':'Desc-1020-5_6-KP_6',
    'Desc-1020-3_4-KP_3':'Desc-1020-5_6-KP_5'
},]

In [3]:
#Set Sheet = all
Data=[]
sheet_list=pd.ExcelFile(IO).sheet_names


In [4]:
Stuff=PSD(io=IO,sheet_name='Case')


In [5]:
df=Stuff.psd_data

In [ ]:
# for old_pn, new_pn in find_remove[0].items():
#     df.replace(to_replace=old_pn, value=new_pn, inplace=True, regex=True)
#     print('Old PN is:'+old_pn,'\nNew PN is:'+new_pn)

In [ ]:
pd.ExcelFile(IO).sheet_names

In [14]:
#Writing Replacement Function

def find_and_replace(df:pd.DataFrame,to_replace,new_value)->None:
    df.replace(to_replace=to_replace,value=new_value,regex=True,inplace=True)

    

In [17]:
import pandas as pd
from utils.Dataframe_tools import PSD 
from utils import add_function_to_task,process_dir,multi_PSD,write_new_PSD

MYDIR= r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files"
target_dir=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files'

#Setting the Values
IO='input files\KPBom.xlsx'

find_remove=[{
    '91840054':'99717392',
    '91840056':'99717485',
    '98441969':'92758523',
    '96698939':'92758528',     # in KPbom.xlsx 
    '96698968':'92758121',     # in KPbom.xlsx
    '96768950':'92758767',     # in KPbom.xlsx
    '99413027':'92758120',
    '1020-3/4':'1020-5/6',
    '1020-3_4':'1020-5_6',
    'Desc-1020-3_4-KP_4':'Desc-1020-5_6-KP_6',
    'Desc-1020-3_4-KP_3':'Desc-1020-5_6-KP_5'
},]

def find_and_replace(df:pd.DataFrame,to_replace,new_value)->None:
    df.replace(to_replace=to_replace,value=new_value,regex=True,inplace=True)


tasks=process_dir(MYDIR,[['Case']],target_dir,False)
add_function_to_task(tasks,find_and_replace,list(find_remove[0].keys()),list(find_remove[0].values()))
multi_PSD(tasks,write_new_PSD)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.